In [1]:
!pip install yfinance
import numpy as np
import pandas as pd
import warnings
import os
import yfinance as yf
import matplotlib.pylab as plt

     |████████████████████████████████| 6.4 MB 5.3 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
A = pd.read_excel('/content/Backtesting(2009~2019).xlsx', dtype={'거래소코드':str})

In [3]:
A

,회사명,거래소코드,Z(K2) Score,F Score,시가총액
0,(주)한진중공업홀딩스,003480,-37.922085,6,76482213080
1,금호전기(주),001210,-33.796449,6,44043971400
2,STX중공업(주),071970,-25.506715,8,85789688205
3,(주)씨아이테크,004920,-22.017654,7,24829240728
4,웰바이오텍(주),010600,-19.829073,6,93977081610
...,...,...,...,...,...
81,(주)에스지글로벌,001380,0.466548,6,77563336425
82,무림피앤피(주),009580,0.542649,6,263506168900
83,동아에스티(주),170900,0.638220,7,1004820292000
84,동일고무벨트(주),163560,0.652069,7,73558500000


In [4]:
A['거래소코드'] = A['거래소코드'].astype(str) + '.KS'

In [5]:
def df2dict(A):
     
    df_dict = []
    
    for k, v in zip(A.iloc[:, 0], A.iloc[:, 1]): # 행 전체 열은 0,1번열 즉 회사명, 거래소코드 라네! .....
        temp = {}
        temp[k]=v
        df_dict.append(temp)

    return df_dict

In [6]:
stocks=df2dict(A)

In [7]:
stocks

[{'(주)한진중공업홀딩스': '003480.KS'},
 {'금호전기(주)': '001210.KS'},
 {'STX중공업(주)': '071970.KS'},
 {'(주)씨아이테크': '004920.KS'},
 {'웰바이오텍(주)': '010600.KS'},
 {'(주)넥스트사이언스': '003580.KS'},
 {'세하(주)': '027970.KS'},
 {'(주)에이엔피': '015260.KS'},
 {'코스모화학(주)': '005420.KS'},
 {'(주)엔케이': '085310.KS'},
 {'동양철관(주)': '008970.KS'},
 {'서울식품공업(주)': '004410.KS'},
 {'세원이앤씨(주)': '091090.KS'},
 {'금호타이어(주)': '073240.KS'},
 {'깨끗한나라(주)': '004540.KS'},
 {'(주)큐로': '015590.KS'},
 {'에이치에스디엔진(주)': '082740.KS'},
 {'효성첨단소재(주)': '298050.KS'},
 {'사조동아원(주)': '008040.KS'},
 {'(주)코아스': '071950.KS'},
 {'(주)조비': '001550.KS'},
 {'(주)동방': '004140.KS'},
 {'효성중공업(주)': '298040.KS'},
 {'(주)세아제강': '306200.KS'},
 {'하이트진로홀딩스(주)': '000140.KS'},
 {'(주)우진플라임': '049800.KS'},
 {'한솔테크닉스(주)': '004710.KS'},
 {'범양건영(주)': '002410.KS'},
 {'일양약품(주)': '007570.KS'},
 {'한화시스템(주)': '272210.KS'},
 {'(주)우진': '105840.KS'},
 {'한솔제지(주)': '213500.KS'},
 {'(주)크라운제과': '264900.KS'},
 {'(주)지에스글로벌': '001250.KS'},
 {'(주)티비에이치글로벌': '084870.KS'},
 {'효성화학(주)': '298000.KS'},


In [8]:
df_1=[]

In [9]:
for stock in stocks :
  for k in stock.keys():
    df = yf.download(stock[k], start = '2019-01-01', end='2019-12-31') #기간변경
    df_1.append(df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [10]:
n = 20 #이평선
sigma = 2 #시그마
base_date = '2009-01-02'

In [11]:
def bollinger_band(price_df, n, sigma):
    bb = price_df.copy()
    bb['center'] = price_df['Adj Close'].rolling(n).mean() #중앙 이동평균선
    bb['ub'] = bb['center'] + sigma * price_df['Adj Close'].rolling(n).std() # 상단 밴드
    bb['lb'] = bb['center'] - sigma * price_df['Adj Close'].rolling(n).std() # 하단 밴드
    return bb

In [12]:
def create_trade_book(sample):
    book = sample[['Adj Close']].copy()
    book['trade'] = ''
    return (book)    

In [13]:
def tradings(sample, book):
    for i in sample.index:
        if sample.loc[i, 'Adj Close'] > sample.loc[i, 'ub']: # 상단밴드 이탈시 동작 안함
            book.loc[i, 'trade'] = ''
        elif sample.loc[i, 'lb'] > sample.loc[i, 'Adj Close']: # 하반밴드 이탈시 매수
            if book.shift(1).loc[i, 'trade'] == 'buy':    # 이미 매수상태라면
                book.loc[i, 'trade'] = 'buy'     # 매수상태 유지
            else:
                book.loc[i, 'trade'] = 'buy'    
        elif sample.loc[i, 'ub'] >= sample.loc[i, 'Adj Close'] and sample.loc[i, 'Adj Close'] >= sample.loc[i, 'lb']: # 볼린저 밴드 안에 있을 시
            if book.shift(1).loc[i, 'trade'] == 'buy':
                book.loc[i, 'trade'] = 'buy'  # 매수상태 유지
            else:
                book.loc[i, 'trade'] = '' # 동작 안함
    return (book) 

In [14]:
def returns(book):
    # 손익 계산
    rtn = 1.0
    book['return'] = 1
    buy = 0.0
    sell = 0.0
    for i in book.index:
        if book.loc[i, 'trade'] == 'buy' and book.shift(1).loc[i, 'trade'] == '':     # long 진입
            buy = book.loc[i, 'Adj Close']
#            print('진입일 : ',i, 'long 진입가격 : ', buy)
        elif book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy':     # long 청산
            sell = book.loc[i, 'Adj Close']
            rtn = (sell - buy) / buy + 1 # 손익 계산
            book.loc[i, 'return'] = rtn
#            print('청산일 : ',i, 'long 진입가격 : ', buy, ' |  long 청산가격 : ', \
#                  sell, ' | return:', round(rtn, 4))
    
        if book.loc[i, 'trade'] == '':     # zero position
            buy = 0.0
            sell = 0.0
    
    acc_rtn = 1.0
    for i in book.index:
        rtn = book.loc[i, 'return']
        acc_rtn = acc_rtn * rtn  # 누적 수익률 계산
        book.loc[i, 'acc return'] = acc_rtn

    print ('Accunulated return :', round(acc_rtn, 4))
    return (round(acc_rtn, 4))

In [15]:
C=[]

In [16]:
for i in range(len(stocks)):
  for comp in stocks[i].keys():
    print(comp)
  df = df_1[i]
  price_df = df.loc[:,['Adj Close']].copy()
  price_df['center'] = price_df['Adj Close'].rolling(20).mean() # 20일 이동평균선
  # +2 표준편차를 둔 상단 밴드
  price_df['ub'] = price_df['center'] + 2 * price_df['Adj Close'].rolling(20).std() 
  # -2 표준편차를 둔 하단 밴드
  price_df['lb'] = price_df['center'] - 2 * price_df['Adj Close'].rolling(20).std()
  bollinger = bollinger_band(price_df, n, sigma)
  sample = bollinger.loc[base_date:]
  book = sample[['Adj Close']].copy()
  book['trade'] = '' #거래내역 컬럼
  book = create_trade_book(sample)
  book = tradings(sample,book)
  

  for i in book.index:
    if book.loc[i, 'trade'] == 'buy' and book.shift(1).loc[i, 'trade'] == '':     # long 진입
        buy = book.loc[i, 'Adj Close']
#        print('진입일 : ',i, 'long 진입가격 : ', buy)
    elif book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy':     # long 청산
        sell = book.loc[i, 'Adj Close']
        rtn = (sell - buy) / buy + 1 # 손익 계산
        book.loc[i, 'return'] = rtn
#        print('청산일 : ',i, 'long 진입가격 : ', buy, ' |  long 청산가격 : ', \
#              sell, ' | return:', round(rtn, 4))
            
    if book.loc[i, 'trade'] == '':     # zero position
        buy = 0.0
        sell = 0.0
  
  C.append(returns(book))
  C
   
#  print(returns(book))

(주)한진중공업홀딩스
Accunulated return : 0.8108
금호전기(주)
Accunulated return : 0.478
STX중공업(주)
Accunulated return : 0.701
(주)씨아이테크
Accunulated return : 1.1035
웰바이오텍(주)
Accunulated return : 0.7497
(주)넥스트사이언스
Accunulated return : 0.8968
세하(주)
Accunulated return : 1.0832
(주)에이엔피
Accunulated return : 0.8026
코스모화학(주)
Accunulated return : 0.4969
(주)엔케이
Accunulated return : 0.7886
동양철관(주)
Accunulated return : 1.0671
서울식품공업(주)
Accunulated return : 1.14
세원이앤씨(주)
Accunulated return : 0.9078
금호타이어(주)
Accunulated return : 0.8655
깨끗한나라(주)
Accunulated return : 0.969
(주)큐로
Accunulated return : 1.2269
에이치에스디엔진(주)
Accunulated return : 0.9131
효성첨단소재(주)
Accunulated return : 1.1153
사조동아원(주)
Accunulated return : 0.8151
(주)코아스
Accunulated return : 1.4413
(주)조비
Accunulated return : 0.76
(주)동방
Accunulated return : 1.0857
효성중공업(주)
Accunulated return : 0.6513
(주)세아제강
Accunulated return : 1.1134
하이트진로홀딩스(주)
Accunulated return : 1.1144
(주)우진플라임
Accunulated return : 0.7476
한솔테크닉스(주)
Accunulated return : 1.422
범양건영(주)
Accunu

In [17]:
C_1=pd.DataFrame(C)
C_1

,0
0,0.8108
1,0.4780
2,0.7010
3,1.1035
4,0.7497
...,...
81,1.2415
82,1.0387
83,0.9728
84,0.8497


In [18]:
C_1.rename(columns = {0 : 'Accunulated return'}, inplace = True) # 열 이름변경
C_1

,Accunulated return
0,0.8108
1,0.4780
2,0.7010
3,1.1035
4,0.7497
...,...
81,1.2415
82,1.0387
83,0.9728
84,0.8497


In [19]:
C_2 = pd.concat([A['회사명'], C_1], axis=1)
C_2 

,회사명,Accunulated return
0,(주)한진중공업홀딩스,0.8108
1,금호전기(주),0.4780
2,STX중공업(주),0.7010
3,(주)씨아이테크,1.1035
4,웰바이오텍(주),0.7497
...,...,...
81,(주)에스지글로벌,1.2415
82,무림피앤피(주),1.0387
83,동아에스티(주),0.9728
84,동일고무벨트(주),0.8497


In [20]:
C_2.to_excel('/content/sample_data/Bollinger Bands for문.xlsx')